In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.cuda.get_device_name(device))

NVIDIA GeForce RTX 3060 Ti


In [2]:
import pandas as pd
df = pd.read_csv('data/data_4000_selected.csv')
df = df.iloc[:, 2:4]
df

,Comment,Majority_Label
0,@User.IDX في فترة الصغر والمراهقة يكون من الصع...,Non-Offensive
1,"""ردا على معظم الردود .. أحب اوضحلكم ان عمليات ...",Non-Offensive
2,@User.IDX يجب ان تذكروا ان لكل سنة ثيم للحفل و...,Non-Offensive
3,بتعمل حلقة صغيرة عشان عندي امتحان بكرة ومتضيعل...,Non-Offensive
4,على طاري السطحيه مدري ليه تذكرت فيحان,Non-Offensive
...,...,...
3995,والله ما اعرف ابكي على حالنا لي وصلنا ليه ام ا...,Non-Offensive
3996,انا الحين ملخبط ذي كلها في البطاقه الائتمانية ...,Non-Offensive
3997,أنا متأكد لو كان ستيفن_هوكينج مسلما وثبتت تجار...,Non-Offensive
3998,على فكرة في الدين حكم الاراهبين يطبق عليهم حكم...,Non-Offensive


In [3]:
test_data = df.sample(frac=0.2,random_state=200)
test_data.shape
data=df.drop(test_data.index)
data.shape,test_data.shape

((3200, 2), (800, 2))

In [4]:
import numpy as np

In [5]:
import emoji
import nltk
import re
import string

arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations = arabic_punctuations + english_punctuations


def remove_urls (text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return text


def remove_emails(text):
    text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", "",  text, flags=re.MULTILINE)
    return text

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def normalization(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def remove_stopwords(text):
    filtered_sentence = [w for w in text.split() if not w in arabic_stopwords]
    return ' '.join(filtered_sentence)

def cleaning_content(line):
    if (isinstance(line, float)):
        return None
    line.replace('\n', ' ')
    line = remove_emails(line)
    line = remove_urls(line)
    line = remove_emoji(line)
    nline = [w if '@' not in w else 'USERID' for w in line.split()]
    line = ' '.join(nline)
    line = line.replace('RT', '').replace('<LF>', '').replace('<br />','').replace('&quot;', '').replace('<url>', '').replace('USERID', '')


    # add spaces between punc,
    line = line.translate(str.maketrans({key: " {0} ".format(key) for key in punctuations}))

    # then remove punc,
    translator = str.maketrans('', '', punctuations)
    line = line.translate(translator)

    line = remove_stopwords(line)
    line=remove_diacritics(normalization(line))
    return line

def hasDigits(s):
    return any( 48 <= ord(char) <= 57  or 1632 <= ord(char) <= 1641 for char in s)


In [6]:
df.Comment = df.Comment.apply(cleaning_content)

In [7]:
comments = ' '.join(list(df.Comment))
words = comments.split(' ')

In [8]:
words

['فتره',
 'الصغر',
 'والمراهقه',
 'يكون',
 'الصعب',
 'تقبل',
 'تعليقات',
 'الاصدقاء',
 'حول',
 'شعرك',
 'المجعد',
 'وهنا',
 'ياتي',
 'دور',
 'الوالدين',
 'ادخال',
 'الثقه',
 'الي',
 'نفسيه',
 'طفلهما',
 'تقدم',
 'بالسن',
 'يصبح',
 'الامر',
 'عاديا',
 'اتكلم',
 'تجربتي',
 'شخصيه',
 'ردا',
 'معظم',
 'الردود',
 'احب',
 'اوضحلكم',
 'ان',
 'عمليات',
 'زرع',
 'الكلي',
 'تتم',
 'الا',
 'في',
 'حالات',
 'معينه',
 'تضمن',
 'عدم',
 'تكرار',
 'الفشل',
 'في',
 'الكليه',
 'المزروعه',
 'وهذا',
 'يحدث',
 'الامراض',
 'الوراثيه',
 'لابد',
 'استبعادها',
 'قبل',
 'بدء',
 'اجراءات',
 'الزرع',
 'فارجو',
 'التواصل',
 'الاطباء',
 'قبل',
 'تجميع',
 'اي',
 'مبلغ',
 'مالي',
 'لعمليه',
 'الزرع',
 'لضمان',
 'وصول',
 'المال',
 'الي',
 'مستحقيه',
 'التواصل',
 'تليفونيا',
 'متاح',
 'حاليا',
 'ويرجي',
 'الذهاب',
 'شخصيا',
 'لمكان',
 'تواجد',
 'الطفل',
 'الطفل',
 'اسمه',
 'محمد',
 'سعد',
 'النادي',
 'موجود',
 'ايام',
 'الاحد',
 'والتلات',
 'والخميس',
 'مستشفي',
 'الدمرداش',
 'مستشفيات',
 'جامعه',
 'عين',
 'شمس',
 'مبن

In [9]:
unigram = (pd.Series(nltk.ngrams(words, 1)).value_counts())[:20]
unigram

(و,)           1232
(الله,)         900
(ان,)           456
(الي,)          383
(انا,)          319
(اللي,)         298
(او,)           245
(مش,)           206
(والله,)        200
(كان,)          193
(انت,)          189
(عمر,)          172
(الا,)          157
(يعني,)         154
(العرب,)        148
(انه,)          146
(علي,)          134
(شي,)           133
(السعوديه,)     133
(قطر,)          131
dtype: int64

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02', do_lower_case=True)

MAX_LEN = 256

def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=cleaning_content(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
        )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [11]:
max_len = 512

# Concatenate train data and test data
all_tweets = np.concatenate([data.texts.values, test_data.texts.values])

# Encode our concatenated data
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

# Find the maximum length
max_len1 = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len1)

AttributeError: 'DataFrame' object has no attribute 'texts'